# L5: Content Creation at Scale

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [2]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [4]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

> Note: The 'llama-3.1-70b-versatil' model was deprecated. As a result, we updated it to the 'llama-3.3-70b-versatil' model in January 2025.

In [5]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama-3.3-70b-versatile"

## Creating Crew, Agents, and Tasks

In [6]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

2025-02-16 15:34:51,177 - 140150507797376 - sqlite.py-sqlite:111 - WARNING: ⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


## Kicking off the Crew

In [7]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: Thought: To create a detailed summary report on the most impactful financial news and updates related to Inflation in the US and the impact on the stock market in 2024, I need to gather information from reliable financial news sources and utilize real-time market data tools. My first step will be to search for relevant news articles and updates on the int



# Agent: Lead Market Analyst
## Thought: content
Thought: I have now gathered a lot of information related to inflation in the US and its impact on the stock market in 2024. It seems that the US inflation rate has been declining since 2022 but still remains above the Federal Reserve's target of 2%. The latest data shows that the headline CPI for the 12 months ending in January 2025 is up to 3.0%. The core CPI, which excludes the volatile food and energy categories, rose 3.3% for the 12 months ending in January. The Federal Reserve has been calibrating its policy to keep inflation in check, and it seems that the Fed is not in a hurry to adjust its policy stance. The impact of inflation on the stock market has been mixed, with some stocks struggling to adjust to the new outlook. It's essential to continue monitoring the situation and analyzing the latest news and updates to provide a detailed summary report.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"infla



# Agent: Lead Market Analyst
## Thought: You must now follow the format requested and use ONLY the allowed tools
Here is the rewritten response:
Thought: Based on the search results and website content, it appears that the US inflation rate has been declining since 2022 but still remains above the Federal Reserve's target of 2%. The latest data shows that the headline CPI for the 12 months ending in January 2025 is up to 3.0%. The core CPI, which excludes the volatile food and energy categories, rose 3.3% for the 12 months ending in January. The Federal Reserve has been calibrating its policy to keep inflation in check, and it seems that the Fed is not in a hurry to adjust its policy stance. The impact of inflation on the stock market has been mixed, with some stocks struggling to adjust to the new outlook.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation rate 2024\"}"
## Tool Output: 

Search results: Title: United States Inflation Rate - Trading

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00, 14.93it/s]




# Agent: Chief Data Strategist
## Thought: Title: Impact of inflation on stock market - VOX, CEPR Policy Portal
Link: https://voxeu.org/article/timing-and-impact-monetary-policy
Snippet: The impact of inflationary shocks on stock prices depends on how fast firms can adjust their production to meet new market conditions.
Thought: Based on the search results, it appears that the current US inflation rate is around 3.0% for the 12 months ending in January 2025, with a core inflation rate of 3.3% for the same period. The Federal Reserve has been calibrating its policy to keep inflation in check, and it seems that the Fed is not in a hurry to adjust its policy stance. The impact of inflation on the stock market has been mixed, with some stocks struggling to adjust to the new outlook.
To gather more information on the impact of inflation on the stock market, I will search a specific website for more details.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"

 Error parsing LLM output, agent will retry: I did it wrong. Invalid Format: I missed the 'Action:' after 'Thought:'. I will do right next, and don't use a tool I have already used.

If you don't need to use any more tools, you must give your best complete final answer, make sure it satisfy the expect criteria, use the EXACT format below:

Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.




# Agent: Chief Data Strategist
## Final Answer: 
Comprehensive Analysis Report: Impact of Inflation on the Stock Market in 2024

Introduction:
The current US inflation rate is around 3.0% for the 12 months ending in January 2025, with a core inflation rate of 3.3% for the same period. The Federal Reserve has been calibrating its policy to keep inflation in check, and it seems that the Fed is not in a hurry to adjust its policy stance. The impact of inflation on the stock market has been mixed, with some stocks struggling to adjust to the new outlook.





# Agent: Chief Content Officer
## Final Answer: 
# Inflation in the US and Its Impact on the Stock Market in 2024

## Understanding Inflation in 2024

Inflation continues to be a significant factor influencing economic conditions in the United States. As we look into 2024, experts predict that inflation rates will experience moderate fluctuations, driven by various elements such as consumer demand, production costs, and government policies. The Federal Reserve's response to these dynamics will also play an essential role in determining inflation levels and their broader implications for the economy.

The recent data suggests that inflation rates are stabilizing following the tumultuous spikes seen during the pandemic years. However, economists note that underlying factors like labor shortages and supply chain disruptions could sustain pressure on prices. As consumers adjust their spending patterns, it will be crucial to monitor how these shifts translate into inflationary trends thro

## Social Content

In [8]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
Inflation in 2024: 🟡 Predictions suggest rates
around 3-4%. How will this affect the stock
market? #Inflation #StockMarket #Investment
--------------------------------------------------
LinkedIn
Understanding how inflation impacts investments is
crucial for 2024. Experts predict ongoing
fluctuations. Is your portfolio ready? #Finance
#Investing #Inflation
--------------------------------------------------
Facebook
As inflation stabilizes, it's essential to assess
your investment strategies! Join us in exploring
the stock market's response in 2024.
#EconomicConditions #InvestSmart
--------------------------------------------------
Instagram
📈🔍 Navigating inflation and the stock market in
2024! Are you prepared? Check out our latest
insights! #InvestmentTips #Inflation2024
#StockMarket
--------------------------------------------------


## Blog Post

In [9]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

# Inflation in the US and Its Impact on the Stock Market in 2024

## Understanding Inflation in 2024

Inflation continues to be a significant factor influencing economic conditions in the United States. As we look into 2024, experts predict that inflation rates will experience moderate fluctuations, driven by various elements such as consumer demand, production costs, and government policies. The Federal Reserve's response to these dynamics will also play an essential role in determining inflation levels and their broader implications for the economy.

The recent data suggests that inflation rates are stabilizing following the tumultuous spikes seen during the pandemic years. However, economists note that underlying factors like labor shortages and supply chain disruptions could sustain pressure on prices. As consumers adjust their spending patterns, it will be crucial to monitor how these shifts translate into inflationary trends throughout the year.

## The Stock Market's Reaction to Inflation

Stock markets typically react to inflation signals, often viewing rising prices as a precursor to increased interest rates. In 2024, investors must remain vigilant as the Federal Reserve may tighten monetary policy to combat inflation, potentially leading to market volatility. Historical perspectives indicate that periods of high inflation can pressure corporate margins and investment valuations, creating a challenging environment for equities.

Moreover, sectors may respond differently to inflationary pressures. For example, energy and utilities often perform better during inflationary periods, while technology stocks might face headwinds as borrowing costs rise. Thus, understanding sectoral performances will be key for investors looking to navigate the stock market in 2024 effectively.

## Key Insights from Analysts

### Inflation Projections

- **Projected Rate**: Analysts anticipate inflation to hover around 3-4% in 2024, slightly above the Federal Reserve's target.
- **Consumer Behavior**: Expect ongoing shifts in consumer spending due to sustained price pressures, impacting overall economic growth.

### Stock Market Strategy

- **Diversification**: Investors are advised to diversify portfolios to mitigate risks associated with sector-specific downturns during inflationary periods.
- **Watch Indicators**: Keep an eye on key indicators such as producer price index (PPI) and consumer price index (CPI) for signals of inflation trends.

## Recommended Actions for Investors

1. **Evaluate Investment Portfolio**: Assess your current investment allocation to ensure it aligns with potential inflationary impacts. Consider increasing exposure to sectors traditionally resilient to inflation.
   
2. **Stay Informed**: Continuously educate yourself on economic conditions through reliable financial news sources and reports. Staying informed will enhance your ability to make timely investment decisions.

## Conclusion

As we progress through 2024, navigating the effects of inflation and its undeniable impact on the stock market requires adaptability and informed strategies. With the potential for volatility and sector-specific reactions, investors must approach their financial decisions with a keen awareness of both economic indicators and market dynamics. As always, consulting with financial advisors can provide additional guidance tailored to individual investment goals.

[Read more about investment strategies during inflation](https://www.financialwebsite.com/investment-strategies-inflation). 

Stay alert, stay informed, and make educated investment decisions to thrive in this changing economic landscape.